In [4]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import h5py
from torch.utils.data import Dataset, DataLoader
import numpy as np
import preprocessing

In [13]:
train_dir = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train/"
train_file_names = os.listdir(train_dir)

In [14]:
train_file_names

['train_10.h5',
 'train_5.h5',
 'train_0.h5',
 'train_8.h5',
 'train_14.h5',
 'train_4.h5',
 'train_1.h5',
 'train_6.h5',
 'train_12.h5',
 'train_7.h5',
 'train_9.h5',
 'train_2.h5',
 'train_13.h5',
 'train_11.h5',
 'train_3.h5']

In [35]:
class H5Dataset(Dataset):
    def __init__(self, file_paths, batch_size, transform=None):
        self.file_paths = file_paths
        self.batch_size = batch_size
        self.transform = transform
        self.sample_indices = []  # Store the indices of samples within each file
        self.FEATURE_KEYS = ['fjet_clus_eta', 'fjet_clus_phi', 'fjet_clus_pt', 'fjet_clus_E']

        for file_path in file_paths:
            with h5py.File(file_path, 'r') as file:
                num_samples = len(file[self.FEATURE_KEYS[0]])  # 'data' is the dataset name in your HDF5 file
                indices = list(range(num_samples))
                self.sample_indices.extend([(file_path, idx) for idx in indices])

    def __len__(self):
        return len(self.sample_indices)

    def __getitem__(self, idx):
        file_path, sample_idx = self.sample_indices[idx]
        
        with h5py.File(file_path, 'r') as file:
            data = file['data'][sample_idx]  # Load a single sample

        if self.transform:
            data = self.transform(data)

        return data

In [36]:
dataset = H5Dataset([train_dir + filename for filename in train_file_names], batch_size=256)

In [38]:
dataset

KeyError: "Unable to open object (object 'data' doesn't exist)"

In [22]:
f = h5py.File(train_dir + train_file_names[0], 'r')

In [24]:
f.keys()

<KeysViewHDF5 ['fjet_clus_E', 'fjet_clus_eta', 'fjet_clus_phi', 'fjet_clus_pt', 'fjet_eta', 'fjet_m', 'fjet_phi', 'fjet_pt', 'labels', 'training_weights']>

In [34]:
f["fjet_clus_E"]

<HDF5 dataset "fjet_clus_E": shape (5000000, 200), type "<f4">